In [1]:
import transformers
import pandas as pd
# from datasets import load_dataset, load_dataset_builder, Dataset
import torch
import importlib
import numpy as np
import torch.utils.data as tud
import torch.nn as nn
from tqdm import tqdm

In [2]:
import sys
sys.path.append("../../library")
from juan import *
import utils
import juan

In [3]:
data = pd.read_csv("../../data/task_1.csv")
show(data)

(3817, 2)


,sentence,task_1
0,La solución es desarrollar el pensamiento crít...,False
1,Hay que enseñar que la magia no existe.,False
2,Que todos los avances de la humanidad siempre ...,False
3,Enseñar en las escuelas la historia de las rel...,False
4,Desde las religiones de la edad de piedra hast...,False


In [4]:
# %%time
# tokenizer = transformers.AutoTokenizer.from_pretrained("beto/tokenizer")

In [5]:
# tokenizer.save_pretrained("beto/tokenizer/")

In [6]:
# %%time
# model = transformers.AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels = 2)

In [7]:
# model.save_pretrained("beto/model")

In [8]:
%%time
tokenizer = transformers.AutoTokenizer.from_pretrained("../../assets/beto/tokenizer")

CPU times: user 102 ms, sys: 45.4 ms, total: 147 ms
Wall time: 287 ms


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [12]:
model = transformers.AutoModel.from_pretrained("../../assets/beto/model")
model.to(device)

X = data.sentence.head(100)
y = data.task_1.to_frame(100)

importlib.reload(juan)
importlib.reload(utils)
clf = juan.Beto(tokenizer, model)
clf.to(device)
# clf.entrenar(
#     data.sentence.head(100), 
#     data.task_1.head(100), 
#     epochs = 5,
# #     freeze_encoder = True,
# #     weight_decay = 1000,
#     class_weights = "balanced"
# )

# predictions = clf.predecir(data.sentence)

results = utils.validate(
    data.sentence.head(100), 
    data.task_1.to_frame().head(100), 
    clf, 
    epochs = 5,
#     freeze_encoder = True,
    progress_bar = False,
#     weight_decay = 100,
    class_weights = "balanced"
)
results

Some weights of the model checkpoint at ../../assets/beto/model were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Cross-validation process started...
*** column 1 / 1 (task_1) - fold 1 / 5
    training model...
tensor([ 1.0959, 11.4286], device='cuda:0')
      first step, train loss:1.2101
      epoch:   1, train loss:0.5357, train accuracy: 0.8625
      epoch:   2, train loss:0.1926, train accuracy: 0.9625
      epoch:   3, train loss:0.1057, train accuracy: 0.9750
      epoch:   4, train loss:0.0455, train accuracy: 1.0000
      epoch:   5, train loss:0.0157, train accuracy: 1.0000
    generating predictions on the train set...
    generating predictions on the test set...
    Total runtime: 0.11 minutes
*** column 1 / 1 (task_1) - fold 2 / 5
    training model...
tensor([ 1.0959, 11.4286], device='cuda:0')
      first step, train loss:1.0417
      epoch:   1, train loss:0.5938, train accuracy: 0.8500
      epoch:   2, train loss:0.2200, train accuracy: 0.9625
      epoch:   3, train loss:0.0769, train accuracy: 0.9875
      epoch:   4, train loss:0.0296, train accuracy: 1.0000
      epoch:   5,

,test_accuracy,test_f1,test_precision,test_recall,train_accuracy,train_f1,train_precision,train_recall
column,,,,,,,,
task_1,0.94,0.333333,0.4,0.3,1.0,1.0,1.0,1.0
